# 📊 Telco Customer Churn - Exploratory Data Analysis

**Author:** Vũ Tiến Đức  
**Email:** vutienduc.31032003@gmail.com  
**Date:** February 2025

---

## 📋 Table of Contents
1. [Import Libraries & Load Data](#1-import-libraries--load-data)
2. [Data Overview](#2-data-overview)
3. [Data Cleaning](#3-data-cleaning)
4. [Univariate Analysis](#4-univariate-analysis)
5. [Bivariate Analysis](#5-bivariate-analysis)
6. [Correlation Analysis](#6-correlation-analysis)
7. [Key Insights](#7-key-insights)

## 1. Import Libraries & Load Data

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings

# Settings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

# Color palette
colors = {
    'primary': '#1E40AF',
    'secondary': '#059669',
    'accent': '#DC2626',
    'churn': '#EF4444',
    'retain': '#10B981'
}

print("✅ Libraries imported successfully!")

In [ ]:
# Load dataset
df = pd.read_csv('../data/cleaned_telco_customer_churn.csv')

print(f"Dataset loaded successfully!")
print(f"Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")

## 2. Data Overview

In [ ]:
# First look at the data
df.head(10)

In [ ]:
# Data types and info
print("=" * 50)
print("DATA TYPES & INFO")
print("=" * 50)
df.info()

In [ ]:
# Statistical summary for numerical columns
print("=" * 50)
print("STATISTICAL SUMMARY - NUMERICAL COLUMNS")
print("=" * 50)
df.describe()

In [ ]:
# Categorical columns summary
print("=" * 50)
print("CATEGORICAL COLUMNS SUMMARY")
print("=" * 50)

cat_cols = df.select_dtypes(include=['object']).columns.tolist()
for col in cat_cols:
    print(f"\n{col}:")
    print(df[col].value_counts())

## 3. Data Cleaning

In [ ]:
# Check for missing values
print("=" * 50)
print("MISSING VALUES CHECK")
print("=" * 50)

missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Missing %': missing_pct
})
print(missing_df[missing_df['Missing Count'] > 0])

if missing.sum() == 0:
    print("\n✅ No missing values found!")

In [ ]:
# Check for duplicates
print("=" * 50)
print("DUPLICATES CHECK")
print("=" * 50)

duplicates = df.duplicated().sum()
print(f"Duplicate rows: {duplicates}")

duplicate_ids = df['customerID'].duplicated().sum()
print(f"Duplicate customerIDs: {duplicate_ids}")

if duplicates == 0 and duplicate_ids == 0:
    print("\n✅ No duplicates found!")

In [ ]:
# Convert TotalCharges to numeric (if needed)
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# Create binary Churn column for analysis
df['Churn_Binary'] = df['Churn'].map({'Yes': 1, 'No': 0})

# Create tenure buckets
df['tenure_bucket'] = pd.cut(df['tenure'], 
                              bins=[0, 6, 12, 24, 48, 72],
                              labels=['0-6', '7-12', '13-24', '25-48', '49-72'])

# Create Monthly Charges buckets
df['charges_bucket'] = pd.cut(df['MonthlyCharges'],
                               bins=[0, 35, 55, 75, 95, 120],
                               labels=['$0-35', '$35-55', '$55-75', '$75-95', '$95+'])

print("✅ Feature engineering completed!")

## 4. Univariate Analysis

In [ ]:
# Target variable distribution - CHURN
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Count plot
churn_counts = df['Churn'].value_counts()
ax1 = axes[0]
bars = ax1.bar(churn_counts.index, churn_counts.values, 
               color=[colors['retain'], colors['churn']], edgecolor='white', linewidth=2)
ax1.set_title('Customer Churn Distribution', fontsize=14, fontweight='bold')
ax1.set_xlabel('Churn Status')
ax1.set_ylabel('Number of Customers')

# Add value labels
for bar, count in zip(bars, churn_counts.values):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 50,
             f'{count:,}', ha='center', fontsize=12, fontweight='bold')

# Pie chart
ax2 = axes[1]
churn_pct = df['Churn'].value_counts(normalize=True) * 100
ax2.pie(churn_pct, labels=['Retained', 'Churned'], autopct='%1.1f%%',
        colors=[colors['retain'], colors['churn']], explode=[0, 0.05],
        startangle=90, textprops={'fontsize': 12})
ax2.set_title('Churn Rate', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('../reports/churn_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n📊 CHURN SUMMARY:")
print(f"   - Total Customers: {len(df):,}")
print(f"   - Churned: {churn_counts['Yes']:,} ({churn_pct['Yes']:.1f}%)")
print(f"   - Retained: {churn_counts['No']:,} ({churn_pct['No']:.1f}%)")

In [ ]:
# Numerical features distribution
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

numerical_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']
titles = ['Customer Tenure (months)', 'Monthly Charges ($)', 'Total Charges ($)']

for ax, col, title in zip(axes, numerical_cols, titles):
    # Histogram
    ax.hist(df[col].dropna(), bins=30, color=colors['primary'], 
            edgecolor='white', alpha=0.7)
    ax.axvline(df[col].mean(), color=colors['accent'], linestyle='--', 
               linewidth=2, label=f'Mean: {df[col].mean():.2f}')
    ax.axvline(df[col].median(), color=colors['secondary'], linestyle='-', 
               linewidth=2, label=f'Median: {df[col].median():.2f}')
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.set_xlabel(col)
    ax.set_ylabel('Frequency')
    ax.legend()

plt.tight_layout()
plt.savefig('../reports/numerical_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Key categorical features distribution
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

cat_features = ['Contract', 'InternetService', 'PaymentMethod', 
                'TechSupport', 'gender', 'SeniorCitizen']

for ax, col in zip(axes, cat_features):
    counts = df[col].value_counts()
    bars = ax.bar(range(len(counts)), counts.values, color=colors['primary'], 
                  edgecolor='white', alpha=0.8)
    ax.set_xticks(range(len(counts)))
    ax.set_xticklabels(counts.index, rotation=45, ha='right')
    ax.set_title(f'{col} Distribution', fontsize=11, fontweight='bold')
    ax.set_ylabel('Count')
    
    # Add value labels
    for bar, count in zip(bars, counts.values):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 20,
                f'{count:,}', ha='center', fontsize=9)

plt.tight_layout()
plt.savefig('../reports/categorical_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

## 5. Bivariate Analysis - Churn vs Features

In [ ]:
# Function to calculate churn rate by category
def churn_rate_by_category(df, column):
    return df.groupby(column)['Churn_Binary'].agg(['mean', 'count']).reset_index()

# Churn rate by Contract type
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# 1. Contract
ax = axes[0, 0]
contract_churn = churn_rate_by_category(df, 'Contract')
bars = ax.bar(contract_churn['Contract'], contract_churn['mean'] * 100,
              color=[colors['churn'] if x > 0.3 else colors['primary'] for x in contract_churn['mean']])
ax.set_title('Churn Rate by Contract Type', fontsize=12, fontweight='bold')
ax.set_ylabel('Churn Rate (%)')
ax.axhline(y=df['Churn_Binary'].mean() * 100, color='gray', linestyle='--', label='Overall Avg')
for bar, rate in zip(bars, contract_churn['mean']):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
            f'{rate*100:.1f}%', ha='center', fontweight='bold')
ax.legend()

# 2. Internet Service
ax = axes[0, 1]
internet_churn = churn_rate_by_category(df, 'InternetService')
bars = ax.bar(internet_churn['InternetService'], internet_churn['mean'] * 100,
              color=[colors['churn'] if x > 0.3 else colors['primary'] for x in internet_churn['mean']])
ax.set_title('Churn Rate by Internet Service', fontsize=12, fontweight='bold')
ax.set_ylabel('Churn Rate (%)')
ax.axhline(y=df['Churn_Binary'].mean() * 100, color='gray', linestyle='--', label='Overall Avg')
for bar, rate in zip(bars, internet_churn['mean']):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
            f'{rate*100:.1f}%', ha='center', fontweight='bold')
ax.legend()

# 3. Payment Method
ax = axes[1, 0]
payment_churn = churn_rate_by_category(df, 'PaymentMethod')
bars = ax.barh(payment_churn['PaymentMethod'], payment_churn['mean'] * 100,
               color=[colors['churn'] if x > 0.3 else colors['primary'] for x in payment_churn['mean']])
ax.set_title('Churn Rate by Payment Method', fontsize=12, fontweight='bold')
ax.set_xlabel('Churn Rate (%)')
ax.axvline(x=df['Churn_Binary'].mean() * 100, color='gray', linestyle='--', label='Overall Avg')
for bar, rate in zip(bars, payment_churn['mean']):
    ax.text(bar.get_width() + 0.5, bar.get_y() + bar.get_height()/2,
            f'{rate*100:.1f}%', va='center', fontweight='bold')
ax.legend()

# 4. Tenure Bucket
ax = axes[1, 1]
tenure_churn = churn_rate_by_category(df, 'tenure_bucket')
bars = ax.bar(tenure_churn['tenure_bucket'].astype(str), tenure_churn['mean'] * 100,
              color=[colors['churn'] if x > 0.3 else colors['primary'] for x in tenure_churn['mean']])
ax.set_title('Churn Rate by Tenure', fontsize=12, fontweight='bold')
ax.set_ylabel('Churn Rate (%)')
ax.set_xlabel('Tenure (months)')
ax.axhline(y=df['Churn_Binary'].mean() * 100, color='gray', linestyle='--', label='Overall Avg')
for bar, rate in zip(bars, tenure_churn['mean']):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
            f'{rate*100:.1f}%', ha='center', fontweight='bold')
ax.legend()

plt.tight_layout()
plt.savefig('../reports/churn_by_category.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Numerical features vs Churn
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# 1. Tenure distribution by Churn
ax = axes[0]
for churn_status, color in [('No', colors['retain']), ('Yes', colors['churn'])]:
    subset = df[df['Churn'] == churn_status]['tenure']
    ax.hist(subset, bins=20, alpha=0.6, label=f'Churn={churn_status}', color=color)
ax.set_title('Tenure Distribution by Churn', fontweight='bold')
ax.set_xlabel('Tenure (months)')
ax.set_ylabel('Frequency')
ax.legend()

# 2. Monthly Charges by Churn (Boxplot)
ax = axes[1]
df.boxplot(column='MonthlyCharges', by='Churn', ax=ax,
           patch_artist=True,
           boxprops=dict(facecolor='lightblue'))
ax.set_title('Monthly Charges by Churn', fontweight='bold')
ax.set_xlabel('Churn')
ax.set_ylabel('Monthly Charges ($)')
plt.suptitle('')

# 3. Total Charges by Churn (Boxplot)
ax = axes[2]
df.boxplot(column='TotalCharges', by='Churn', ax=ax,
           patch_artist=True,
           boxprops=dict(facecolor='lightgreen'))
ax.set_title('Total Charges by Churn', fontweight='bold')
ax.set_xlabel('Churn')
ax.set_ylabel('Total Charges ($)')
plt.suptitle('')

plt.tight_layout()
plt.savefig('../reports/numerical_vs_churn.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Statistical Tests
print("=" * 60)
print("STATISTICAL TESTS - Numerical Features vs Churn")
print("=" * 60)

for col in ['tenure', 'MonthlyCharges', 'TotalCharges']:
    churned = df[df['Churn'] == 'Yes'][col].dropna()
    retained = df[df['Churn'] == 'No'][col].dropna()
    
    # T-test
    t_stat, p_value = stats.ttest_ind(churned, retained)
    
    print(f"\n{col}:")
    print(f"   Churned Mean: {churned.mean():.2f}")
    print(f"   Retained Mean: {retained.mean():.2f}")
    print(f"   T-statistic: {t_stat:.4f}")
    print(f"   P-value: {p_value:.4e}")
    print(f"   Significant: {'✅ Yes' if p_value < 0.05 else '❌ No'}")

## 6. Correlation Analysis

In [ ]:
# Create correlation matrix for numerical features
# First, encode categorical variables
df_encoded = df.copy()

# Binary encoding
binary_cols = ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']
for col in binary_cols:
    df_encoded[col] = df_encoded[col].map({'Yes': 1, 'No': 0})

df_encoded['gender'] = df_encoded['gender'].map({'Male': 1, 'Female': 0})

# Select numerical columns for correlation
corr_cols = ['Churn_Binary', 'tenure', 'MonthlyCharges', 'TotalCharges', 
             'SeniorCitizen', 'Partner', 'Dependents', 'PaperlessBilling']

corr_matrix = df_encoded[corr_cols].corr()

# Plot correlation matrix
plt.figure(figsize=(10, 8))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r',
            center=0, square=True, linewidths=1)
plt.title('Correlation Matrix - Key Features', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('../reports/correlation_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Correlation with Churn
print("=" * 50)
print("CORRELATION WITH CHURN (sorted)")
print("=" * 50)

churn_corr = corr_matrix['Churn_Binary'].drop('Churn_Binary').sort_values(key=abs, ascending=False)
print(churn_corr)

## 7. Key Insights

In [ ]:
# Summary Statistics
print("="*70)
print("📊 KEY INSIGHTS SUMMARY")
print("="*70)

print("\n🎯 OVERALL CHURN:")
print(f"   • Total Customers: {len(df):,}")
print(f"   • Churn Rate: {df['Churn_Binary'].mean()*100:.1f}%")
print(f"   • Revenue at Risk: ${df[df['Churn']=='Yes']['MonthlyCharges'].sum():,.2f}/month")

print("\n⚠️ HIGH-RISK SEGMENTS:")
print(f"   • Month-to-month Contract: {df[df['Contract']=='Month-to-month']['Churn_Binary'].mean()*100:.1f}% churn")
print(f"   • Fiber Optic Internet: {df[df['InternetService']=='Fiber optic']['Churn_Binary'].mean()*100:.1f}% churn")
print(f"   • Electronic Check Payment: {df[df['PaymentMethod']=='Electronic check']['Churn_Binary'].mean()*100:.1f}% churn")
print(f"   • New Customers (≤6 months): {df[df['tenure']<=6]['Churn_Binary'].mean()*100:.1f}% churn")

print("\n✅ LOW-RISK SEGMENTS:")
print(f"   • Two-year Contract: {df[df['Contract']=='Two year']['Churn_Binary'].mean()*100:.1f}% churn")
print(f"   • Long-tenure (>48 months): {df[df['tenure']>48]['Churn_Binary'].mean()*100:.1f}% churn")

print("\n💡 KEY FINDINGS:")
print("   1. Contract type is the strongest predictor of churn")
print("   2. New customers (first 6 months) have highest churn risk")
print("   3. Fiber optic customers churn more than DSL customers")
print("   4. Lack of add-on services correlates with higher churn")
print("   5. Electronic check users have significantly higher churn")

In [ ]:
# Save cleaned dataset for modeling
df.to_csv('../data/processed_telco_churn.csv', index=False)
print("✅ Processed dataset saved to '../data/processed_telco_churn.csv'")